In [197]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus.util import remove_stopwords


In [198]:
rstate=10

In [199]:
df = pd.read_csv("Bengali hate speech.csv")

In [200]:
import string 
def remove_punctuation_from_sentence(sentence):
    sentence_without_punctuation="".join([c for c in sentence if c not in string.punctuation])
    return sentence_without_punctuation

df['sentence without punctuation'] = df['sentence'].apply(lambda x:remove_punctuation_from_sentence(x))
df.head()

,sentence,hate,category,sentence without punctuation
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব


In [201]:
df['Removed Stopped word'] = df['sentence without punctuation'].apply(lambda x: remove_stopwords(x,stopwords))
df.head()

,sentence,hate,category,sentence without punctuation,Removed Stopped word
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,"[যত্তসব, পাপন, শালার, ফাজলামী]"
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,"[পাপন, শালা, রে, রিমান্ডে, দরকার]"
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,"[জিল্লুর, রহমান, স্যারের, ছেলে, এতো, বড়, জারজ,..."
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,"[শালা, লুচ্চা, পাঠার, মত, যায়]"
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,"[তুই, শালা, গাজা, খাইছচ, ।, তুর, মার, হেডায়, খ..."


In [202]:
df['hate_speech']=df['Removed Stopped word'].apply(' '.join)

In [203]:
y=df['hate']

In [204]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['hate_speech'])
sequences = tokenizer.texts_to_sequences(df['hate_speech'])
x= pad_sequences(sequences, maxlen=100)

In [205]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=rstate)

In [206]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=100))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [207]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [214]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy*100.0)

Epoch 1/5
750/750 [==============================] - 67s 87ms/step - loss: 0.0495 - accuracy: 0.9825 - val_loss: 0.5730 - val_accuracy: 0.8458
Epoch 2/5
750/750 [==============================] - 65s 86ms/step - loss: 0.0397 - accuracy: 0.9853 - val_loss: 0.6931 - val_accuracy: 0.8483
Epoch 3/5
750/750 [==============================] - 66s 88ms/step - loss: 0.0327 - accuracy: 0.9887 - val_loss: 0.6979 - val_accuracy: 0.8462
Epoch 4/5
750/750 [==============================] - 67s 90ms/step - loss: 0.0251 - accuracy: 0.9910 - val_loss: 0.6966 - val_accuracy: 0.8442
Epoch 5/5
188/188 [==============================] - 2s 11ms/step - loss: 0.8512 - accuracy: 0.8327
Test Loss: 0.8512223362922668
Test Accuracy: 83.26666951179504


In [215]:
#Saving the model
model_json=model.to_json()
with open("bangla-sequential-model.json","w")as json_file:
    json_file.write(model_json)
model.save_weights("bangla-sequential-model_wieghts.h5")

In [216]:
model.save("bangla-sequential-model.h5")

In [217]:
from tensorflow.keras.models import load_model

loaded_model = load_model(r"bangla-sequential-model.h5")

In [212]:
text="আমি ভালো আছি বেশ"
text=remove_punctuation_from_sentence(text)
text=remove_stopwords(text=text,stopwords=stopwords)

In [213]:
tokenizer = Tokenizer()
    
 
sequences = tokenizer.texts_to_sequences([text])
pdsequences= pad_sequences(sequences, maxlen=100)
prediction = loaded_model.predict(pdsequences)[0][0]
predicted_class = (prediction > 0.5).astype(int)
print(predicted_class)

1/1 [==============================] - 0s 203ms/step
0
